In [14]:
'''https://stmorse.github.io/journal/spotify-api.html'''

import json
import requests

In [15]:
'''' load credentials via json'''
with open('credentials.json', 'r') as r:
    credentials = json.load(r)

In [16]:
''' generate auth token and headers'''
AUTH_URL = 'https://accounts.spotify.com/api/token'
# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': credentials['client_id'],
    'client_secret': credentials['client_secret'],
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

# generate header
headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

In [19]:
# base URL of all Spotify API endpoints
BASE_URL = 'https://api.spotify.com/v1/'
# Track ID from the URI
track_id = '6y0igZArWVi6Iz0rj35c1Y'
# actual GET request with proper header
r = requests.get(BASE_URL + 'audio-features/' + track_id, headers=headers)
r.json()

{'danceability': 0.54,
 'energy': 0.59,
 'key': 0,
 'loudness': -4.359,
 'mode': 1,
 'speechiness': 0.0528,
 'acousticness': 0.446,
 'instrumentalness': 0,
 'liveness': 0.14,
 'valence': 0.267,
 'tempo': 119.878,
 'type': 'audio_features',
 'id': '6y0igZArWVi6Iz0rj35c1Y',
 'uri': 'spotify:track:6y0igZArWVi6Iz0rj35c1Y',
 'track_href': 'https://api.spotify.com/v1/tracks/6y0igZArWVi6Iz0rj35c1Y',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/6y0igZArWVi6Iz0rj35c1Y',
 'duration_ms': 234910,
 'time_signature': 4}

In [22]:
artist_id = '36QJpDe2go2KgaRleHCDTp'

# pull all artists albums
r = requests.get(BASE_URL + 'artists/' + artist_id + '/albums', 
                 headers=headers, 
                 params={'include_groups': 'album', 'limit': 50})
d = r.json()

dict

In [41]:
print(d.keys())
d['items'][0].keys()
# type(d['items'][0])
# d

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])


dict_keys(['album_group', 'album_type', 'artists', 'available_markets', 'external_urls', 'href', 'id', 'images', 'name', 'release_date', 'release_date_precision', 'total_tracks', 'type', 'uri'])

In [42]:
data = []   # will hold all track info
albums = [] # to keep track of duplicates

# loop over albums and get all tracks
for album in d['items']:
    album_name = album['name']

    # here's a hacky way to skip over albums we've already grabbed
    trim_name = album_name.split('(')[0].strip()
    if trim_name.upper() in albums or int(album['release_date'][:4]) > 1983:
        continue
    albums.append(trim_name.upper()) # use upper() to standardize
    
    # this takes a few seconds so let's keep track of progress    
    print(album_name)
    
    # pull all tracks from this album
    r = requests.get(BASE_URL + 'albums/' + album['id'] + '/tracks', 
        headers=headers)
    tracks = r.json()['items']
    
    for track in tracks:
        # get audio features (key, liveness, danceability, ...)
        f = requests.get(BASE_URL + 'audio-features/' + track['id'], 
            headers=headers)
        f = f.json()
        
        # combine with album info
        f.update({
            'track_name': track['name'],
            'album_name': album_name,
            'short_album_name': trim_name,
            'release_date': album['release_date'],
            'album_id': album['id']
        })
        
        data.append(f)

Coda (Deluxe Edition)
In Through the out Door (Deluxe Edition)
The Song Remains the Same (Remaster)
Presence (Deluxe Edition)
Physical Graffiti (1994 Remaster)
Houses of the Holy (Deluxe Edition)
Led Zeppelin IV (Deluxe Edition)
Led Zeppelin III (Deluxe Edition)
Led Zeppelin II (Deluxe Edition)
Led Zeppelin (Deluxe Edition)


In [49]:
data[0].keys()

dict_keys(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature', 'track_name', 'album_name', 'short_album_name', 'release_date', 'album_id'])

In [50]:
for x in data:
    print(x['track_name'],x['album_name'])

We're Gonna Groove - Remaster Coda (Deluxe Edition)
Poor Tom - Remaster Coda (Deluxe Edition)
I Can't Quit You Baby - Remaster Coda (Deluxe Edition)
Walter's Walk - Remaster Coda (Deluxe Edition)
Ozone Baby - Remaster Coda (Deluxe Edition)
Darlene - Remaster Coda (Deluxe Edition)
Bonzo's Montreux - Remaster Coda (Deluxe Edition)
Wearing and Tearing - Remaster Coda (Deluxe Edition)
We're Gonna Groove - Alternate Mix Coda (Deluxe Edition)
If It Keeps on Raining - Rough Mix Coda (Deluxe Edition)
Bonzo's Montreux - Mix Construction in Progress Coda (Deluxe Edition)
Baby Come on Home - Remastered Coda (Deluxe Edition)
Sugar Mama - Mix Coda (Deluxe Edition)
Poor Tom - Instrumental Mix Coda (Deluxe Edition)
Travelling Riverside Blues (BBC Session) - Remastered Coda (Deluxe Edition)
Hey, Hey, What Can I Do - Remaster Coda (Deluxe Edition)
Four Hands (Four Sticks) [Bombay Orchestra] Coda (Deluxe Edition)
Friends (Bombay Orchestra) Coda (Deluxe Edition)
St. Tristan's Sword - Rough Mix Coda (Delu